### Load forest inventory data from database ('Bundeswaldinventur 2012' https://www.bundeswaldinventur.de/)

In [25]:
import pandas as pd
import geopandas as gpd
import psycopg2

# PostGIS connection credentials
user = 'dc_user'
password = 'localuser1234'
host = '127.0.0.1'
port = '5432'
database = 'datacube'


In [26]:
connection = psycopg2.connect(user = user,
                              password = password,
                              host = host,
                              port = port,
                              database = database)

table =pd.read_sql_query("SELECT * FROM information_schema.columns WHERE table_schema = 'public';",con=connection )
connection.close()
table

# to have a look to all entries, use following to show in the notebook
#pd.set_option("display.max_rows", None, "display.max_columns", None)
#table
# or
# write a to csv table to the dir of the used notebook
#table.to_csv('./table.csv')

,table_catalog,table_schema,table_name,column_name,ordinal_position,column_default,is_nullable,data_type,character_maximum_length,character_octet_length,...,is_identity,identity_generation,identity_start,identity_increment,identity_maximum,identity_minimum,identity_cycle,is_generated,generation_expression,is_updatable
0,datacube,public,auth_group,id,1,nextval('auth_group_id_seq'::regclass),NO,integer,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
1,datacube,public,auth_group,name,2,None,NO,character varying,80.0,320.0,...,NO,None,None,None,None,None,NO,NEVER,None,YES
2,datacube,public,accounts_activation,username,1,None,NO,character varying,30.0,120.0,...,NO,None,None,None,None,None,NO,NEVER,None,YES
3,datacube,public,accounts_activation,url,2,None,NO,uuid,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
4,datacube,public,accounts_activation,time,3,None,NO,timestamp with time zone,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
880,datacube,public,demmin_landnutzung_2017,cg_id,7,None,YES,smallint,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
881,datacube,public,demmin_landnutzung_2017,cc_id,8,None,YES,smallint,NaN,NaN,...,NO,None,None,None,None,None,NO,NEVER,None,YES
882,datacube,public,demmin_landnutzung_2017,sub_class,9,None,YES,character varying,25.0,100.0,...,NO,None,None,None,None,None,NO,NEVER,None,YES
883,datacube,public,demmin_landnutzung_2017,crop_pract,10,None,YES,character varying,50.0,200.0,...,NO,None,None,None,None,None,NO,NEVER,None,YES


In [28]:
# check db entry by name, here 'etrs89_bwi_2012p'
# 1. open connection first, 2. write data, 3. close afterwards
connection = psycopg2.connect(user = user,
                              password = password,
                              host = host,
                              port = port,
                              database = database)
# write attribute names
tables =pd.read_sql_query("SELECT  column_name FROM information_schema.columns WHERE table_name = 'etrs89_bwi_2012p';",con=connection )
connection.close()
# show attributes of table 'etrs89_bwi_2012p'
tables

,column_name
0,gid
1,tnr
2,mitrand
3,anzhbbeg
4,phaupt
5,pohaupt
6,pquerus
7,pfagus
8,pfraxinus
9,pcarpinus


In [29]:
# check percent share of plot 'plarix' ('p' percent share, 'larix' lärche )
connection = psycopg2.connect(user = user,
                              password = password,
                              host = host,
                              port = port,
                              database = database)

table =pd.read_sql_query("SELECT  plarix FROM etrs89_bwi_2012p;",con=connection)
connection.close()
table

,plarix
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
2837,0.0
2838,0.0
2839,0.0
2840,0.0


In [30]:
# show betula areas with > 50 % share
table.loc[table['plarix'] > 0.50]

,plarix
33,0.743
172,0.580
466,1.000
478,0.542
643,0.507
691,0.779
964,0.541
1093,1.000
1967,0.719
2075,0.968


In [32]:
# load db entry into shapefile
# db connection
connection = psycopg2.connect(database=database, 
                              user=user, 
                              password=password, 
                              host=host)

# choose attributes, e.g. when comparing 
# table =pd.read_sql_query("SELECT  column_name FROM information_schema.columns WHERE table_name = 'etrs89_bwi_2012p';",con=connection )
# with 
# inventory metadata (/Datacube/Forest inventory data/MetadatenFuerTraktdaten.csv for layer-name 'GFs_ReinbestandGattung_mitSp_PolyVar')


sql_command = "SELECT geom, pquerus, pfagus, pfraxinus FROM etrs89_bwi_2012p;"
forest = gpd.GeoDataFrame.from_postgis(sql_command, connection) 
connection.close()

forest

,geom,pquerus,pfagus,pfraxinus
0,"MULTIPOLYGON (((597233.574 5241596.664, 597233...",0.000,0.313,0.000
1,"MULTIPOLYGON (((595210.408 5243571.865, 595210...",0.000,0.097,0.000
2,"MULTIPOLYGON (((599205.778 5243622.840, 599205...",0.000,0.000,0.000
3,"MULTIPOLYGON (((593187.241 5245547.065, 593187...",0.000,0.000,0.000
4,"MULTIPOLYGON (((597182.612 5245598.041, 597182...",0.000,0.513,0.121
...,...,...,...,...
2837,"MULTIPOLYGON (((620849.021 5575961.846, 620849...",0.697,0.024,0.000
2838,"MULTIPOLYGON (((660823.100 5577490.897, 660823...",0.000,0.000,0.000
2839,"MULTIPOLYGON (((622875.212 5573988.618, 622875...",0.000,0.000,0.000
2840,"MULTIPOLYGON (((626926.597 5570043.163, 626926...",0.000,0.000,0.000


In [17]:
connection = psycopg2.connect(database=database, 
                              user=user, 
                              password=password, 
                              host=host)

sql_command = "SELECT * FROM etrs89_bwi_2012p WHERE pquerus >= 0.8 ;"
forest = gpd.GeoDataFrame.from_postgis(sql_command, connection) 
connection.close()

forest


,gid,tnr,mitrand,anzhbbeg,phaupt,pohaupt,pquerus,pfagus,pfraxinus,pcarpinus,...,peurop,pdecid,pconifer,pall,cell_id,epsg,netz,dichte,bl,geom
0,398,1648,1,1,1.000,0.000,1.000,0.000,0.000,0.000,...,1.000,1.000,0.000,1.000,4486_2747,31468,16,16,;9;,"MULTIPOLYGON (((739078.965 5300024.456, 739078..."
1,425,1911,1,1,1.000,0.000,0.897,0.000,0.000,0.000,...,1.000,1.000,0.000,1.000,4506_2752,31468,16,16,;9;,"MULTIPOLYGON (((759012.947 5305259.232, 759012..."
2,975,6727,1,1,1.000,0.000,0.949,0.000,0.000,0.000,...,1.000,1.000,0.000,1.000,4532_2829,31468,16,16,;9;,"MULTIPOLYGON (((784048.156 5382620.909, 784048..."
3,1020,7211,1,1,1.000,0.000,0.811,0.000,0.000,0.189,...,1.000,1.000,0.000,1.000,4380_2833,31468,16,8,;9;,"MULTIPOLYGON (((632362.472 5385096.354, 632362..."
4,1041,7260,1,1,1.000,0.000,1.000,0.000,0.000,0.000,...,1.000,1.000,0.000,1.000,4576_2838,31468,16,16,;9;,"MULTIPOLYGON (((827955.456 5392140.846, 827955..."
5,1087,7733,0,1,1.000,0.000,0.921,0.000,0.000,0.079,...,1.000,1.000,0.000,1.000,4396_2841,31468,16,16,;9;,"MULTIPOLYGON (((647918.273 5392969.108, 647918..."
6,1089,7736,1,2,1.000,0.000,0.889,0.000,0.000,0.111,...,1.000,1.000,0.000,1.000,4408_2841,31468,16,16,;9;,"MULTIPOLYGON (((659911.412 5393119.991, 659911..."
7,1153,8232,1,1,1.000,0.000,1.000,0.000,0.000,0.000,...,1.000,1.000,0.000,1.000,4351_2848,31467,16,4,;8;,"MULTIPOLYGON (((603420.191 5399959.587, 603420..."
8,1197,8488,1,1,1.000,0.000,0.806,0.194,0.000,0.000,...,1.000,1.000,0.000,1.000,4380_2853,31468,256,8,;9;,"MULTIPOLYGON (((632101.667 5405098.312, 632101..."
9,1238,8780,1,1,1.000,0.000,1.000,0.000,0.000,0.000,...,1.000,1.000,0.000,1.000,4568_2862,31468,16,16,;9;,"MULTIPOLYGON (((819646.948 5416070.167, 819646..."


In [33]:
# get extents of shape 
x_min = forest.bounds.iloc[0,0]
x_max = forest.bounds.iloc[0,2]
y_min = forest.bounds.iloc[0,1]
y_max = forest.bounds.iloc[0,3]
# transform to lon/lat for plotting if needed
forest_wgs = forest.to_crs('EPSG:4326')
longitude_extents = (forest_wgs.bounds.iloc[0,0],forest_wgs.bounds.iloc[0,2])
latitude_extents  = (forest_wgs.bounds.iloc[0,1],forest_wgs.bounds.iloc[0,3])

forest_wgs.bounds

,minx,miny,maxx,maxy
0,10.286685,47.320376,10.308198,47.335003
1,10.260342,47.338447,10.281856,47.353070
2,10.313227,47.338302,10.334755,47.352934
3,10.233981,47.356513,10.255495,47.371130
4,10.286885,47.356380,10.308413,47.371007
...,...,...,...,...
2837,10.697715,50.323023,10.720692,50.337731
2838,11.259526,50.327104,11.282667,50.341916
2839,10.725524,50.304861,10.748500,50.319574
2840,10.781063,50.268525,10.804038,50.283249
